In [6]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV 

import nltk 
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
import pickle
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Creating a list for columns to keep
cols = ['EventDescription','FailedAssets','IncidentCause','IncidentConsequence','IncidentType','WeatherStation','Status','Category']

In [8]:
# Importing file
df = pd.read_csv('cleaned_incidents1.csv', usecols=cols)
print(df.shape)

#dropping nulls
df = df.dropna()
df.isnull().sum()

(6504, 8)


EventDescription       0
FailedAssets           0
IncidentCause          0
IncidentConsequence    0
IncidentType           0
Status                 0
WeatherStation         0
Category               0
dtype: int64

In [9]:
df['Description'] = df['WeatherStation'] + ' ' + df['IncidentType'] + ' ' + df['Status'] + ' ' + df['EventDescription'] + ' ' + df['FailedAssets'] + ' ' + df['IncidentCause']+ ' ' + df['IncidentConsequence']

In [10]:
print(df['Description'])

0       Avalon Airport Infrastructure (network-based) ...
1       Laverton Raaf Infrastructure (network-based) R...
2       Warrnambool Airport Ndb Infrastructure (networ...
3       Essendon Airport Infrastructure (network-based...
4       Avalon Airport Infrastructure (network-based) ...
                              ...                        
6499    Moorabbin Airport Infrastructure (network-base...
6500    Avalon Airport Infrastructure (network-based) ...
6501    Mildura Airport Infrastructure (network-based)...
6502    Swan Hill Aerodrome Infrastructure (network-ba...
6503    Moorabbin Airport Infrastructure (network-base...
Name: Description, Length: 6488, dtype: object



Stopwords, Splitting, Label Encoding

In [11]:
# Creating stopwords list

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
numerical_symbols = re.compile('0-90-9a-z')
 
STOPWORDS = set(stopwords.words('english'))
 
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = numerical_symbols.sub('', text)
    text = text.replace('x', '')
    #text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['Description'] = df['Description'].apply(clean_text)


'''
def getLemmText(text):
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    
    #ps = PorterStemmer()
    #tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
df['Description'] = list(map(getLemmText,df['Description']))


def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
df['Description'] = list(map(getStemmText,df['Description']))

'''

"\ndef getLemmText(text):\n    tokens=word_tokenize(text)\n    lemmatizer = WordNetLemmatizer()\n    tokens=[lemmatizer.lemmatize(word) for word in tokens]\n    \n    #ps = PorterStemmer()\n    #tokens=[ps.stem(word) for word in tokens]\n    return ' '.join(tokens)\ndf['Description'] = list(map(getLemmText,df['Description']))\n\n\ndef getStemmText(text):\n    tokens=word_tokenize(text)\n    ps = PorterStemmer()\n    tokens=[ps.stem(word) for word in tokens]\n    return ' '.join(tokens)\ndf['Description'] = list(map(getStemmText,df['Description']))\n\n"

In [12]:
print(df['Description'])

0       avalon airport infrastructure networkbased rep...
1       laverton raaf infrastructure networkbased repo...
2       warrnambool airport ndb infrastructure network...
3       essendon airport infrastructure networkbased r...
4       avalon airport infrastructure networkbased rep...
                              ...                        
6499    moorabbin airport infrastructure networkbased ...
6500    avalon airport infrastructure networkbased rep...
6501    mildura airport infrastructure networkbased re...
6502    swan hill aerodrome infrastructure networkbase...
6503    moorabbin airport infrastructure networkbased ...
Name: Description, Length: 6488, dtype: object


In [13]:
# Label encoding for Category
le = LabelEncoder()
df['Category'] = le.fit_transform(df['Category'].astype(str))
#store the 'Category' variable in Y
X = df[['Description']]
Y = df[['Category']]

In [46]:
# Splitting of data in test and train
x_train, x_test, y_train, y_test = train_test_split(df['Description'],Y, 
                                                    test_size=0.2, random_state=4)

Naive Bayes

In [47]:
stopwords = nltk.corpus.stopwords.words('english')

In [48]:
vectorizer = TfidfVectorizer(stop_words=stopwords, analyzer='word')
tfidf = vectorizer.fit(df['Description'])

In [49]:
x_train_tfidf =  tfidf.transform(x_train)
x_test_tfidf =  tfidf.transform(x_test)

In [68]:
from sklearn.naive_bayes import ComplementNB

clf = ComplementNB()
# xtrain = x_train_tfidf.todense()

clf.fit(x_train_tfidf, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [69]:
# Model evaluation
# xtest = x_test_tfidf.todense()
prediction = clf.predict(x_test_tfidf)
acc = accuracy_score(y_test, prediction).round(4)
print("Accuracy using TF-IDF is: {}%".format(acc * 100.0))

Accuracy using TF-IDF is: 73.42%
